In [2]:
split_data_tp1_app = open_file('../data/data_tp1_app.txt')
split_data_tp1_dec = open_file('../data/data_tp1_dec.txt')

split_data_tp2_app = open_file('../data/data_tp2_app.txt')
split_data_tp2_dec = open_file('../data/data_tp2_dec.txt')

split_data_tp3_app = open_file('../data/data_tp3_app.txt')
split_data_tp3_dec = open_file('../data/data_tp3_dec.txt')

In [4]:
print("First Data TP1")
start_time = time.time()
best_k = get_k_cross_validation(split_data_tp1_app, 15, 5)
compute_kppv(split_data_tp1_app, split_data_tp1_dec, best_k[0])
print("\tThe best k used is {} with a error rate = {}".format(best_k[0], best_k[1]))
print("\tExecution time : %s seconds\n" % (time.time() - start_time))
print("==========================\n")

print("First Data TP2")
start_time = time.time()
best_k = get_k_cross_validation(split_data_tp2_app, 15, 5)
compute_kppv(split_data_tp2_app, split_data_tp2_dec, best_k[0])
print("\tThe best k used is {} with a error rate = {}".format(best_k[0], best_k[1]))
print("\tExecution time : %s seconds\n" % (time.time() - start_time))
print("==========================\n")

print("First Data TP3")
start_time = time.time()
best_k = get_k_cross_validation(split_data_tp3_app, 15, 5)
compute_kppv(split_data_tp3_app, split_data_tp3_dec, best_k[0])
print("\tThe best k used is {} with a error rate = {}".format(best_k[0], best_k[1]))
print("\tExecution time : %s seconds\n" % (time.time() - start_time))
print("==========================\n")

First Data TP1
	Results :
	----------------
	Number of elements for the learning step :  500
	Number of elements for the decision step :  500
	----------------

	Top results :
	----------------
	Top 1 rate :  0.644
	Top 2 rate :  0.934
	----------------

	Confusion matrix :
	----------------
╒════╤═════╤═════╤═════╤═════╤═════╕
│    │   1 │   2 │   3 │   4 │   5 │
╞════╪═════╪═════╪═════╪═════╪═════╡
│  1 │  48 │  51 │   0 │   0 │   1 │
├────┼─────┼─────┼─────┼─────┼─────┤
│  2 │  45 │  52 │   0 │   0 │   3 │
├────┼─────┼─────┼─────┼─────┼─────┤
│  3 │   0 │   0 │  73 │  27 │   0 │
├────┼─────┼─────┼─────┼─────┼─────┤
│  4 │   0 │   0 │  41 │  55 │   4 │
├────┼─────┼─────┼─────┼─────┼─────┤
│  5 │   1 │   1 │   0 │   4 │  94 │
╘════╧═════╧═════╧═════╧═════╧═════╛
	----------------
	The best k used is 5 with a error rate = 0.626
	Execution time : 8.369944095611572 seconds


First Data TP2
	Results :
	----------------
	Number of elements for the learning step :  500
	Number of elements f